![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

<H1>Context Based Clinical Spell Checker</H1>

In [0]:
from johnsnowlabs import nlp, medical

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
#nlp.install()

In [0]:
from pyspark.sql import DataFrame
import pyspark.sql.functions as F
import pyspark.sql.types as T
import pyspark.sql as SQL

import os
import json
import string
import numpy as np
import pandas as pd

from pyspark.ml import Pipeline, PipelineModel

pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 100)  
pd.set_option('display.expand_frame_repr', False)

spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.0 
 Master 
 spark://10.139.64.10:7077 
 AppName 
 Databricks Shell

## spellcheck_clinical

In [0]:
documentAssembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = nlp.RecursiveTokenizer()\
    .setInputCols(["document"])\
    .setOutputCol("token")\
    .setPrefixes(["\"", "(", "[", "\n"])\
    .setSuffixes([".", ",", "?", ")","!", "'s"])

spellModel = nlp.ContextSpellCheckerModel.pretrained('spellcheck_clinical', 'en', 'clinical/models')\
    .setInputCols("token")\
    .setOutputCol("checked")

spellcheck_clinical download started this may take some time.
Approximate size to download 134.7 MB
[OK!]


In [0]:
pipeline = nlp.Pipeline(
    stages = [
    documentAssembler,
    tokenizer,
    spellModel
  ])

empty_ds = spark.createDataFrame([[""]]).toDF("text")

lp = nlp.LightPipeline(pipeline.fit(empty_ds))

Ok!, at this point we have our spell checking pipeline as expected. Let's see what we can do with it, see these errors,



_She was **treathed** with a five day course of **amoxicilin** for a **resperatory** **truct** infection._

_With pain well controlled on **orall** **meditation**, she was discharged to **reihabilitation** **facilitay**._


_Her **adominal** examination is soft, nontender, and **nonintended**_

_The patient was seen by the **entocrinology** service and she was discharged on 40 units of **unsilin** glargine at night_
      
_No __cute__ distress_


Check that some of the errors are valid English words, only by considering the context the right choice can be made.

In [0]:
example = ["She was treathed with a five day course of amoxicilin for a resperatory truct infection . ",
           "With pain well controlled on orall meditation, she was discharged to reihabilitation facilitay.",
           "Her adominal examination is soft, nontender, and nonintended.",
           "The patient was seen by the entocrinology service and she was discharged on 40 units of unsilin glargine at night",
           "No cute distress",
          ]

for pairs in lp.annotate(example):

  print (list(zip(pairs['token'],pairs['checked'])))

[('She', 'She'), ('was', 'was'), ('treathed', 'treated'), ('with', 'with'), ('a', 'a'), ('five', 'five'), ('day', 'day'), ('course', 'course'), ('of', 'of'), ('amoxicilin', 'amoxicillin'), ('for', 'for'), ('a', 'a'), ('resperatory', 'respiratory'), ('truct', 'tract'), ('infection', 'infection'), ('.', '.')]
[('With', 'With'), ('pain', 'pain'), ('well', 'well'), ('controlled', 'controlled'), ('on', 'on'), ('orall', 'oral'), ('meditation', 'medication'), (',', ','), ('she', 'she'), ('was', 'was'), ('discharged', 'discharged'), ('to', 'to'), ('reihabilitation', 'rehabilitation'), ('facilitay', 'facility'), ('.', '.')]
[('Her', 'Her'), ('adominal', 'abdominal'), ('examination', 'examination'), ('is', 'is'), ('soft', 'soft'), (',', ','), ('nontender', 'nontender'), (',', ','), ('and', 'and'), ('nonintended', 'nondistended'), ('.', '.')]
[('The', 'The'), ('patient', 'patient'), ('was', 'was'), ('seen', 'seen'), ('by', 'by'), ('the', 'the'), ('entocrinology', 'endocrinology'), ('service', 'se

In [0]:
print("Corrected tokens:\n")

pair_list = [list(zip(pairs['token'],pairs['checked'])) for pairs in lp.annotate(example)]
corrected_list = [i for pair in pair_list for i in pair if i[0] != i[1]]
corrected_list

Corrected tokens:

Out[6]: [('treathed', 'treated'),
 ('amoxicilin', 'amoxicillin'),
 ('resperatory', 'respiratory'),
 ('truct', 'tract'),
 ('orall', 'oral'),
 ('meditation', 'medication'),
 ('reihabilitation', 'rehabilitation'),
 ('facilitay', 'facility'),
 ('adominal', 'abdominal'),
 ('nonintended', 'nondistended'),
 ('entocrinology', 'endocrinology'),
 ('unsilin', 'insulin'),
 ('cute', 'acute')]

## spellcheck_drug_norvig

In [0]:
documentAssembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = nlp.Tokenizer()\
    .setInputCols("document")\
    .setOutputCol("token")

spell = nlp.NorvigSweetingModel.pretrained("spellcheck_drug_norvig", "en", "clinical/models")\
    .setInputCols("token")\
    .setOutputCol("corrected_token")\

pipeline = nlp.Pipeline(
    stages = [
        documentAssembler,
        tokenizer,
        spell
        ])


empty_ds = spark.createDataFrame([[""]]).toDF("text")

lp = nlp.LightPipeline(pipeline.fit(empty_ds))

spellcheck_drug_norvig download started this may take some time.
Approximate size to download 4.3 MB
[OK!]


In [0]:
example = ["You have to take Amrosia artemisiifoli , Oactra and a bit of Grastk and lastacaf ",
          ]

for pairs in lp.annotate(example):
    print(list(zip(pairs['token'],pairs['corrected_token'])))

[('You', 'You'), ('have', 'have'), ('to', 'to'), ('take', 'take'), ('Amrosia', 'Ambrosia'), ('artemisiifoli', 'artemisiifolia'), (',', ','), ('Oactra', 'Odactra'), ('and', 'and'), ('a', 'a'), ('bit', 'bit'), ('of', 'of'), ('Grastk', 'Grastek'), ('and', 'and'), ('lastacaf', 'lastacaft')]


In [0]:
print("Corrected tokens:\n")

pair_list = [list(zip(pairs['token'],pairs['corrected_token'])) for pairs in lp.annotate(example)]
corrected_list = [i for pair in pair_list for i in pair if i[0] != i[1]]
corrected_list

Corrected tokens:

Out[9]: [('Amrosia', 'Ambrosia'),
 ('artemisiifoli', 'artemisiifolia'),
 ('Oactra', 'Odactra'),
 ('Grastk', 'Grastek'),
 ('lastacaf', 'lastacaft')]

End of Notebook #